In [1]:
import pandas as pd
import re

# 1. Cargar el dataset (asumiendo que está en la ruta ./data/)
df_moving = pd.read_csv('./data/digital-nomad-index-movingto-clean.csv')

# 2. Definir la función de limpieza
def procesar_impuestos(texto):
    """
    Entrada: String sucio (ej: "0-35%", "NHR 20%", "15%")
    Salida: Serie con (tax_min, tax_max)
    """
    # Si el dato es nulo o no es texto, devolvemos nulos
    if pd.isna(texto):
        return pd.Series([None, None])
    
    # Usamos Regex para encontrar todos los números (incluyendo decimales)
    # \d+ busca dígitos, \.? busca un punto opcional
    numeros = re.findall(r"[\d\.]+", str(texto))
    
    # Convertimos la lista de textos a números flotantes
    numeros = [float(n) for n in numeros]
    
    if len(numeros) == 2:
        # Caso rango: "0-35%" -> [0, 35]
        return pd.Series([numeros[0], numeros[1]])
    elif len(numeros) == 1:
        # Caso valor único: "15%" -> [15] -> Asignamos el mismo a min y max
        return pd.Series([numeros[0], numeros[0]])
    else:
        # Caso raro (sin números o texto incomprensible)
        return pd.Series([None, None])

# 3. Aplicar la función
# Esto crea dos columnas nuevas automáticamente
df_moving[['tax_min', 'tax_max']] = df_moving['taxes'].apply(procesar_impuestos)

# 4. Verificación
print("--- Antes vs Después ---")
print(df_moving[['taxes', 'tax_min', 'tax_max']].head(10))

# 5. (Opcional) Eliminar la columna vieja 'taxes' si ya no la quieres
df_moving = df_moving.drop(columns=['taxes'])

# 6. Guardar el resultado mejorado
df_moving.to_csv('./data/digital-nomad-index-movingto-clean-v2.csv', index=False)

--- Antes vs Después ---
      taxes  tax_min  tax_max
0   NHR 20%    20.00     20.0
1     0-20%     0.00     20.0
2        1%     1.00      1.0
3       24%    24.00     24.0
4     0-35%     0.00     35.0
5  1.92-35%     1.92     35.0
6       15%    15.00     15.0
7     0-30%     0.00     30.0
8       24%    24.00     24.0
9     0-25%     0.00     25.0
